Quelques précisions :
- Dans un premier temps le modèle ne s'entraînera pas à prédire seulement les métiers présent dans la phrase, mais tous les mots. Pour prédire seulement les métiers il faudrait soit des phrases où le métier est toujours placé à la même position, comme ca on peut demander au modèle de mask input[index] mais ca contraint énormément le type de phrase. Soit on cherche un algo automatique qui déduit les tokens en rapport avec les métiers (mais ca prend du temps à chercher/implémenter donc on va commencer sans). Sinon il faudrait avoir une liste de métier et on demande à les masquer mais ca contraint le type de métier parce qu'on ne pensera pas à tout. Par exemple un LLM avait généré comme phrase d'entrée (elle est capitaine de l'équipe de basket professionelle ect) Quels tokens masquer ici?
- Pour l'instant la fonction de coût utilisée est une MSE entre output d'un mot versus la distribution de probabilité qu'on voudrait avoir et qui correspond à la moyenne des outputs pour phrase homme et phrase femme. A cela on ajoute un terme de régularisation entre logits 1 et logits 2 pour les contraindre à être à peu près similaire. (très fortement sujet à débat)

In [ ]:
import torch
import torch.nn as nn
from transformers import Trainer

In [ ]:
# class CustomTrainer(Trainer):
#     def compute_loss(self, model, inputs, return_outputs=False):
#         outputs_1 = model(input_ids=inputs["input_ids_1"], attention_mask=inputs["attention_mask_1"])
#         logits1 = outputs_1.logits

#         outputs_2 = model(input_ids=inputs["input_ids_2"], attention_mask=inputs["attention_mask_2"])
#         logits2 = outputs_2.logits

#         # loss1 = F.cross_entropy(logits1.view(-1, logits1.size(-1)), logits2.size(-1))  # Peut être qu'il vaudrait mieux faire la mean squared error ?
#         # loss2 = F.cross_entropy(logits2.view(-1, logits2.size(-1)), logits1.size(-1))

#         loss1.backward(retain_graph=True)
#         loss2.backward()

#         return (combined_loss, outputs_1) if return_outputs else combined_loss

In [ ]:
class CustomDataCollector(DataCollatorForLanguageModeling) :
  def__init__(self):
    super.__init__()
    self.ground_truth = precompute_loss(model, inputs)

  def torch_call(features): # On override torch call parce que sinon la méthode renvoit un dictionnaire avec les clés {input_ids, attention_mask, label}
      return {
        "input_ids_1": torch.stack([f["input_ids_1"] for f in features]),
        "attention_mask_1": torch.stack([f["attention_mask_1"] for f in features]),
        "input_ids_2": torch.stack([f["input_ids_2"] for f in features]),
        "attention_mask_2": torch.stack([f["attention_mask_2"] for f in features]),
        "ground_truth" : self.ground_truth # si je n'arrive pas à accéder à ground_truth depuis cette method, soit override __init__ de DataCollator soit passer et en faire un attribu
        # ground_truth en argument torch_call
      }

In [ ]:
if not hasaatr(self.ground_truth):
  self.ground_truth = (logits1 + logits2)/ 2
  # Si logits1 et logits2 sont des tensors de taille (batch_size, vocab_size)
  # Ground truth est aussi un tensor de taille (batch_size, sequence_lenght, vocab_size)
  # on ne peut pas app
  # Pour les garder il faudrait les placer dans labels, mais les labels n'auront pas la forme qu'ils sont sencé avoir

def precompute_labels(model, inputs) :
  # return a dictionnary input with one keywords added the ground_truth
  input_1 = inputs["inputs_ids_1"]
  input_2 = inputs["inputs_ids_2"]
  logits1 = model(input_1) # vérifier que model(input_1) ne renvoie que l'output et rien d'autre sinon ca va créer une erreur
  logits2 = model(input_2) # vérifier qu'on appelle bien forward de cet manière et qu'on peut lui passer en input un tensor au lieu d'un dictionnaire

  assert logits1.shape == (batch_size, sequence_length vocab_size)
  ground_truth = (logits1 + logits) /2
  # inputs['ground_truth'] = ground_truth
  # return inputs
  return ground_truth

ground_truth = precompute_labels(model, inputs)

  # On a applique un forward pass sur tout le dataset et on met dans le dictionnaire des inputs mais pas sous le keyword label
  # Limite : peut être très couteux computationnellemnt.
  # La fonction de coût sera la MSE pour que ca soit plus simple même si c'est peut être pas le mieux.

In [ ]:
class Biais_trainer(Trainer):
  def __init__(self):
    super.__init__()
    self.custom_loss = nn.MSELoss()

  def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
    input_1 = inputs["inputs_ids_1"]
    input_2 = inputs["inputs_ids_2"]
    ground_truth = inputs["ground_truth"]
    logits1 = model(input_1) # vérifier que model(input_1) ne renvoie que l'output et rien d'autre sinon ca va créer une erreur (voir avec la doc de Bert)
    logits2 = model(input_2)
    combined_loss = self.custom_loss(logits1, ground_truth) + 0.5* self.custom_loss(logits1, logits2) # le deuxième terme est un terme de similarity constraint
    return (combined_loss, logits_1) if return_outputs else combined_loss

In [ ]:
model = BertForMaskedLM.from_pretrained("bert-base-uncased")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

custom_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path="train.txt",
    block_size=128 # a changer pour qu'on puisse décider à chaque changement de ligne de prendre une autre phrase qui n'a aucun lien.
)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.15)

training_args = TrainingArguments(
    output_dir="./bert-mlm-finetuned",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    save_steps=10_000,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=custom_dataset
)

trainer.train()